# Predict tags on StackOverflow with linear models

In this assignment you will learn how to predict tags for posts from [StackOverflow](https://stackoverflow.com). To solve this task you will use multilabel classification approach.

### Libraries

In this task you will need the following libraries:
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [Pandas](https://pandas.pydata.org) — a library providing high-performance, easy-to-use data structures and data analysis tools for the Python
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [NLTK](http://www.nltk.org) — a platform to work with natural language.

### Data

The following cell will download all data required for this assignment into the folder `week1/data`.

In [ ]:
import sys
sys.path.append("..")
from common.download_utils import download_week1_resources

download_week1_resources()

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('NLP2').getOrCreate()

### Text preprocessing

For this and most of the following assignments you will need to use a list of stop words. It can be downloaded from *nltk*:

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In this task you will deal with a dataset of post titles from StackOverflow. You are provided a split to 3 sets: *train*, *validation* and *test*. All corpora (except for *test*) contain titles of the posts and corresponding tags (100 tags are available). The *test* set is provided for Coursera's grading and doesn't contain answers. Upload the corpora using *pandas* and look at the data:

In [ ]:
train = spark.read.csv('data/train.tsv',inferSchema=True,header=True,sep='\t')
validation=spark.read.csv('data/validation.tsv',inferSchema=True,header=True,sep='\t')
test = spark.read.csv('data/test.tsv',inferSchema=True,header=True,sep='\t')

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from string import ascii_letters, digits

from pyspark.sql.functions import regexp_replace, trim, col, lower
def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        column (Column): A Column containing a sentence.

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
    #df = sqlContext.sql("SELECT * FROM table")
    return lower(trim(regexp_replace(column,'[^A-Za-z0-9 ]+', ''))).alias('sentence')


In [ ]:
#Column title remove punctuation and to lower. respect # and + because of C# and c++
train = train.withColumn('title', lower(regexp_replace(train.title, '[^A-Za-z0-9#+ ]+', ' ')))
validation = validation.withColumn('title', lower(regexp_replace(validation.title, '[^A-Za-z0-9#+ ]+', ' ')))
test = test.withColumn('title', lower(regexp_replace(test.title, '[^A-Za-z0-9#+ ]+', ' ')))


In [ ]:
train.select('title','tags').show()

In [ ]:
# tags in null replace with NoTag 
train = train.withColumn('tags_1', when( train.tags.isNull(), "['NoTag']").otherwise(train.tags))
validation = validation.withColumn('tags_1', when( validation.tags.isNull(), "['NoTag']").otherwise(validation.tags))
#train=train.na.fill('NA')
train.select('tags','tags_1').show(truncate=True, n=290)


In [ ]:
train.filter("tags is NULL").show()

In [ ]:
validation.select('tags','tags_1').show(truncate=True, n=290)

In [ ]:
# Pipeline Tokenizers and remove stop words

regexTokenizer = RegexTokenizer(inputCol="title", outputCol="words", pattern="\\W")
regexTokenizer2 = RegexTokenizer(inputCol="tags_1", outputCol="tags_2", pattern="[^A-Za-z0-9#+]+")

remover = StopWordsRemover(inputCol="words", outputCol="filtered")

train_trans = Pipeline(stages=[regexTokenizer,regexTokenizer2,remover]).fit(train).transform(train)
val_trans = Pipeline(stages=[regexTokenizer,regexTokenizer2, remover]).fit(validation).transform(validation)
test_trans = Pipeline(stages=[regexTokenizer, remover]).fit(test).transform(test)

As you can see, *title* column contains titles of the posts and *tags* colum countains the tags. It could be noticed that a number of tags for a post is not fixed and could be as many as necessary.

For a more comfortable usage, initialize *X_train*, *X_val*, *X_test*, *y_train*, *y_val*.

In [ ]:
X_train=train_trans.select('filtered')
X_val =val_trans.select('filtered')
X_test=test_trans.select('filtered')

One of the most known difficulties when working with natural data is that it's unstructured. For example, if you use it "as is" and extract tokens just by splitting the titles by whitespaces, you will see that there are many "weird" tokens like *3.5?*, *"Flip*, etc. To prevent the problems, it's usually useful to prepare the data somehow. In this task you'll write a function, which will be also used in the other assignments. 

**Task 1 (TextPrepare).** Implement the function *text_prepare* following the instructions. After that, run the function *test_test_prepare* to test it on tiny cases and submit it to Coursera.

In [ ]:
y_train=train_trans.withColumn('tags_1',explode(split('tags',', ')))
y_train=y_train.withColumn('tags_1',regexp_replace(y_train.tags_1, '[^A-Za-z0-9#+ ]+', ''))

y_val=val_trans.withColumn('tags_1',explode(split('tags',', ')))
y_val=y_val.withColumn('tags_1',regexp_replace(y_val.tags_1, '[^A-Za-z0-9#+ ]+', ''))


In [ ]:
y_val.filter("tags is NULL").count()

In [ ]:
y_val.select('filtered','tags','tags_1').show()

In [ ]:
y_train.groupBy("tags") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(n=5)

In [ ]:
y_val.groupBy("tags") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(n=5)

In [ ]:
#create a column label with thag categorized
"""
StringIndexer encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), 
ordered by label frequencies, so the most frequent label gets index 0. The unseen labels will be put at index numLabels 
if user chooses to keep them. If the input column is numeric, we cast it to string and index the string values. 
When downstream pipeline components such as Estimator or Transformer make use of this string-indexed label, you must set 
the input column of the component to this string-indexed column name. In many cases, you can set the input column with 
setInputCol.
"""
label_stringIdx = StringIndexer(inputCol = "tags", outputCol = "label")

dataset = label_stringIdx.fit(y_train).transform(y_train)
dataset2 = label_stringIdx.fit(y_val).transform(y_val)

dataset.show(25)

In [ ]:
dataset2.show(25)

Run your implementation for questions from file *text_prepare_tests.tsv* to earn the points.

Now we can preprocess the titles using function *text_prepare* and  making sure that the headers don't have bad symbols:

For each tag and for each word calculate how many times they occur in the train corpus. 

**Task 2 (WordsTagsCount).** Find 3 most popular tags and 3 most popular words in the train data and submit the results to earn the points.

In [ ]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}

df_dict = X_train.collect()
for r in df_dict:
    for word in r[0]:
        try:
            words_counts[word] += 1
        except KeyError:
            words_counts[word] = 1

#words_counts=sorted(words_counts.items(), key = lambda item: item[1], reverse=True)

df_tags=y_train.select('tags_1').collect()
for r in df_tags:
    for word in r:
        try:
            tags_counts[word] += 1
        except KeyError:
            tags_counts[word] = 1

#tags_counts=sorted(tags_counts.items(), key = lambda item: item[1], reverse=True) 

In [ ]:
print(tags_counts)

We are assume that *tags_counts* and *words_counts* are dictionaries like `{'some_word_or_tag': frequency}`. After appllying the sorting procedure, results will be look like this: `[('most_popular_word_or_tag', frequency), ('less_popular_word_or_tag', frequency), ...]`. The grader gets the results in the following format (two comma-separated strings with line break):

    tag1,tag2,tag3
    word1,word2,word3

Pay attention that in this assignment you should not submit frequencies or some additional information.

In [ ]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]

print('WordsTagsCount', '%s\n%s' % (','.join(tag for tag, _ in most_common_tags), 
                                                ','.join(word for word, _ in most_common_words)))

### Transforming text to a vector

Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

#### Bag of words

One of the well-known approaches is a *bag-of-words* representation. To create this transformation, follow the steps:
1. Find *N* most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2. For each title in the corpora create a zero vector with the dimension equals to *N*.
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

Let's try to do it for a toy example. Imagine that we have *N* = 4 and the list of the most popular words is 

    ['hi', 'you', 'me', 'are']

Then we need to numerate them, for example, like this: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

And we have the text, which we want to transform to the vector:

    'hi how are you'

For this text we create a corresponding zero vector 

    [0, 0, 0, 0]
    
And interate over all words, and if the word is in the dictionary, we increase the value of the corresponding position in the vector:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]

The resulting vector will be 

    [1, 1, 0, 1]
   
Implement the described encoding in the function *my_bag_of_words* with the size of the dictionary equals to 5000. To find the most common words use train data. You can test your code using the function *test_my_bag_of_words*.

In [ ]:
#Bag to words 
DICT_SIZE = 50000
# Input data: Each row is a bag of words with a ID.

# fit a CountVectorizerModel from the corpus TRAIN DATASET
cv = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=DICT_SIZE, minDF=1.0)
model = cv.fit(dataset)
result = model.transform(dataset)

#VALIDATION DATASET
cvv = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=DICT_SIZE, minDF=1.0)
modelv = cvv.fit(dataset2)
resultv = modelv.transform(dataset2)

#result.select('filtered','features').show(truncate=False, n=2)
result.select('features','filtered','label').show()

In [ ]:
resultv.select('features','filtered','label').show()

In [ ]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=DICT_SIZE)
featurizedData = hashingTF.transform(dataset)#TRAIN
featurizedDatav = hashingTF.transform(dataset2)#VALIDATION
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)#TRAIN
idfModelv = idf.fit(featurizedDatav)#VALIDATION
rescaledData = idfModel.transform(featurizedData)#TRAIN
rescaledDatav = idfModelv.transform(featurizedDatav)#VALIDATION

rescaledData.select("filtered", "features",'label').show(truncate=False, n=2)

In [ ]:
rescaledDatav.select("filtered", "features",'label').show(truncate=False, n=2)

As you might notice, we transform the data to sparse representation, to store the useful information efficiently. There are many [types](https://docs.scipy.org/doc/scipy/reference/sparse.html) of such representations, however slkearn algorithms can work only with [csr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) matrix, so we will use this one.

**Task 3 (BagOfWords).** For the 11th row in *X_train_mybag* find how many non-zero elements it has. In this task the answer (variable *non_zero_elements_count*) should be a number, e.g. 20.

#### TF-IDF

The second approach extends the bag-of-words framework by taking into account total frequencies of words in the corpora. It helps to penalize too frequent words and provide better features space. 

Implement function *tfidf_features* using class [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from *scikit-learn*. Use *train* corpus to train a vectorizer. Don't forget to take a look into the arguments that you can pass to it. We suggest that you filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the titles). Also, use bigrams along with unigrams in your vocabulary. 

Implement the function *train_classifier* for training a classifier. In this task we suggest to use One-vs-Rest approach, which is implemented in [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) class. In this approach *k* classifiers (= number of tags) are trained. As a basic classifier, use [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time, because a number of classifiers to train is large.

In [ ]:
train_set=result.select('features','label')
train_val=resultv.select('features','label')
train,test=train_set.randomSplit([0.7,0.3])

In [ ]:
train_val.filter("features is NULL or label is NULL").show()

In [ ]:
print(train.count())
print(test.count())

In [ ]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
    # instantiate the base classifier.
    lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

    # instantiate the One Vs Rest Classifier.
    ovr = OneVsRest(classifier=lr)

    # train the multiclass model.
    ovrModel = ovr.fit(train)

    # score the model on test data.
    predictions = ovrModel.transform(test)

    # obtain evaluator.
    evaluator = MulticlassClassificationEvaluator(metricName="accuracy")



In [ ]:
# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
# $example off$

In [ ]:

# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=12,  maxDepth=5, maxMemoryInMB=1024)

# Chain RF in a Pipeline
pipeline = Pipeline(stages=[rf])

# Train model.
model = rf.fit(train)

# Make predictions.
predictions = model.transform(test)


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC

In [ ]:
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.
    #classifier = OneVsRestClassifier(RidgeClassifier(normalize=True))
    classifier=OneVsRestClassifier(LogisticRegression(penalty="l1", tol=0.0001, C=10))
    #classifier=OneVsRestClassifier(SVC(kernel="linear"))
    classifier.fit(X_train, y_train)
    return classifier

Train the classifiers for different data transformations: *bag-of-words* and *tf-idf*.

In [ ]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

Now you can create predictions for the data. You will need two types of predictions: labels and scores.

In [ ]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

Now take a look at how classifier, which uses TF-IDF, works for a few examples:

In [ ]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(y_val)
for i in range(3):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Now, we would need to compare the results of different predictions, e.g. to see whether TF-IDF transformation helps or to try different regularization techniques in logistic regression. For all these experiments, we need to setup evaluation procedure. 

### Evaluation

To evaluate the results we will use several classification metrics:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
 - [Area under ROC-curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
 - [Area under precision-recall curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) 
 
Make sure you are familiar with all of them. How would you expect the things work for the multi-label scenario? Read about micro/macro/weighted averaging following the sklearn links provided above.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

Implement the function *print_evaluation_scores* which calculates and prints to stdout:
 - *accuracy*
 - *F1-score macro/micro/weighted*
 - *Precision macro/micro/weighted*

In [ ]:
def print_evaluation_scores(y_val, predicted):
    
    print(accuracy_score(y_val, predicted))
    print(f1_score(y_val, predicted, average='weighted'))
    print(average_precision_score(y_val, predicted))

In [ ]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

You might also want to plot some generalization of the [ROC curve](http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) for the case of multi-label classification. Provided function *roc_auc* can make it for you. The input parameters of this function are:
 - true labels
 - decision functions scores
 - number of classes

In [ ]:
from metrics import roc_auc
%matplotlib inline

In [ ]:
n_classes = len(tags_counts)
roc_auc(y_val, y_val_predicted_scores_mybag, n_classes)

In [ ]:
n_classes = len(tags_counts)
roc_auc(y_val, y_val_predicted_scores_tfidf, n_classes)

**Task 4 (MultilabelClassification).** Once we have the evaluation set up, we suggest that you experiment a bit with training your classifiers. We will use *F1-score weighted* as an evaluation metric. Our recommendation:
- compare the quality of the bag-of-words and TF-IDF approaches and chose one of them.
- for the chosen one, try *L1* and *L2*-regularization techniques in Logistic Regression with different coefficients (e.g. C equal to 0.1, 1, 10, 100).

You also could try other improvements of the preprocessing / model, if you want. 

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

When you are happy with the quality, create predictions for *test* set, which you will submit to Coursera.

In [ ]:
test_predictions =classifier_tfidf.predict(X_test_tfidf)
test_pred_inversed = mlb.inverse_transform(test_predictions)

test_predictions_for_submission = '\n'.join('%i\t%s' % (i, ','.join(row)) for i, row in enumerate(test_pred_inversed))
grader.submit_tag('MultilabelClassification', test_predictions_for_submission)

### Analysis of the most important features

Finally, it is usually a good idea to look at the features (words or n-grams) that are used with the largest weigths in your logistic regression model.

Implement the function *print_words_for_tag* to find them. Get back to sklearn documentation on [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) and [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) if needed.

In [ ]:
def print_words_for_tag(classifier, tag, tags_classes, index_to_words, all_words):
    """
        classifier: trained classifier
        tag: particular tag
        tags_classes: a list of classes names from MultiLabelBinarizer
        index_to_words: index_to_words transformation
        all_words: all words in the dictionary
        
        return nothing, just print top 5 positive and top 5 negative words for current tag
    """
    print('Tag:\t{}'.format(tag))
    
    # Extract an estimator from the classifier for the given tag.
    # Extract feature coefficients from the estimator. 
    
    estimator = classifier.estimators_[tags_classes.index(tag)]
    top_positive_words = [index_to_words[index] for index in estimator.coef_.argsort().tolist()[0][-5:]]  # top-5 words sorted by the coefficiens.
    top_negative_words = [index_to_words[index] for index in estimator.coef_.argsort().tolist()[0][:5]] # bottom-5 words  sorted by the coefficients.

    print('Top positive words:\t{}'.format(', '.join(top_positive_words)))
    print('Top negative words:\t{}\n'.format(', '.join(top_negative_words)))

In [ ]:
print_words_for_tag(classifier_tfidf, 'c', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'c++', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'linux', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [ ]:
grader.status()

In [ ]:
STUDENT_EMAIL ='olonok@gmail.com' # EMAIL 
STUDENT_TOKEN = 'bRLk3LTM3vTrz7yI'
grader.status()

If you want to submit these answers, run cell below

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)